In [2]:
from playwright.async_api import async_playwright

p = await async_playwright().start()

In [3]:
browser = await p.chromium.launch(headless=False)

In [4]:
page = await browser.new_page()

In [5]:
await page.goto("https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6")

<Response url='https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6' request=<Request url='https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6' method='GET'>>

In [6]:
textboxes = await page.get_by_role("textbox").all()
textboxes

[<Locator frame=<Frame name= url='https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6'> selector='internal:role=textbox >> nth=0'>,
 <Locator frame=<Frame name= url='https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6'> selector='internal:role=textbox >> nth=1'>]

In [7]:
await page.get_by_placeholder("Search term...").all()

[<Locator frame=<Frame name= url='https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6'> selector='internal:attr=[placeholder="Search term..."i] >> nth=0'>]

In [8]:
element_types: str = "a,button,input"

elements_data = await page.evaluate(f"""
    () => {{
        const elementTypes = '{element_types}'.split(',').map(s => s.trim());
        const selector = elementTypes.join(',');
        const elements = document.querySelectorAll(selector);

        return Array.from(elements).map((el, globalIndex) => {{
            const rect = el.getBoundingClientRect();
            const isVisible = rect.width > 0 && rect.height > 0 &&
                                window.getComputedStyle(el).visibility !== 'hidden' &&
                                window.getComputedStyle(el).display !== 'none';

            // Add a unique data attribute to the element for reliable selection
            const uniqueId = `pw-element-${{globalIndex}}-${{Date.now()}}`;
            el.setAttribute('data-pw-id', uniqueId);

            // Build selector - prefer data attribute for reliability
            let uniqueSelector = `[data-pw-id="${{uniqueId}}"]`;

            // Fallback selectors for reference
            let fallbackSelector = '';
            if (el.id) {{
                fallbackSelector = `#${{el.id}}`;
            }} else {{
                const tagName = el.tagName.toLowerCase();
                // Use XPath-style position for more reliable selection
                const parent = el.parentElement;
                if (parent) {{
                    const allSiblings = Array.from(parent.children);
                    const position = allSiblings.indexOf(el) + 1;

                    if (parent.id) {{
                        fallbackSelector = `#${{parent.id}} > :nth-child(${{position}})`;
                    }} else {{
                        // Get a more specific path
                        fallbackSelector = `${{tagName}}:nth-child(${{position}})`;
                    }}
                }} else {{
                    fallbackSelector = tagName;
                }}
            }}

            return {{
                index: globalIndex,
                tag: el.tagName.toLowerCase(),
                text: el.innerText?.trim().substring(0, 100) || el.value || '',
                href: el.href || null,
                type: el.type || null,
                ariaLabel: el.getAttribute('aria-label'),
                title: el.getAttribute('title'),
                selector: uniqueSelector,
                fallbackSelector: fallbackSelector,
                isVisible: isVisible
            }};
        }}).filter(el => el.isVisible);  // Only return visible elements
    }}
""")

In [9]:
elements_data

[{'index': 0,
  'tag': 'a',
  'text': 'Skip to main content',
  'href': 'https://arxiv.org/search/?query=AI+regulation&searchtype=all&size=50&order=-announced_date_first&start=0&date-filter_by=date&date-year=2022&date-month=6#main-container',
  'type': None,
  'ariaLabel': None,
  'title': None,
  'selector': '[data-pw-id="pw-element-0-1770215130819"]',
  'fallbackSelector': 'a:nth-child(1)',
  'isVisible': True},
 {'index': 1,
  'tag': 'a',
  'text': '',
  'href': 'https://cornell.edu/',
  'type': None,
  'ariaLabel': None,
  'title': None,
  'selector': '[data-pw-id="pw-element-1-1770215130819"]',
  'fallbackSelector': 'a:nth-child(1)',
  'isVisible': True},
 {'index': 2,
  'tag': 'a',
  'text': 'member institutions',
  'href': 'https://info.arxiv.org/about/ourmembers.html',
  'type': None,
  'ariaLabel': None,
  'title': None,
  'selector': '[data-pw-id="pw-element-2-1770215130819"]',
  'fallbackSelector': '#support-ack-url > :nth-child(2)',
  'isVisible': True},
 {'index': 3,
  'ta

In [13]:
text_content = await page.evaluate("""
            () => {
                // Function to get element type and role
                function getElementInfo(el) {
                    const tag = el.tagName.toLowerCase();
                    const role = el.getAttribute('role');
                    const type = el.getAttribute('type');
                    
                    // Determine element category
                    if (tag === 'button' || role === 'button') {
                        return '[BUTTON]';
                    } else if (tag === 'input') {
                        return `[INPUT:${type || 'text'}]`;
                    } else if (tag === 'textarea') {
                        return '[TEXTAREA]';
                    } else if (tag === 'a') {
                        return '[LINK]';
                    } else if (tag === 'article') {
                        return '[ARTICLE]';
                    } else if (tag === 'nav') {
                        return '[NAV]';
                    } else if (tag === 'header') {
                        return '[HEADER]';
                    } else if (tag === 'footer') {
                        return '[FOOTER]';
                    } else if (tag === 'form') {
                        return '[FORM]';
                    } else if (['h1', 'h2', 'h3', 'h4', 'h5', 'h6'].includes(tag)) {
                        return `[${tag.toUpperCase()}]`;
                    } else if (tag === 'select') {
                        return '[SELECT]';
                    } else if (role === 'listbox') {
                        return '[LISTBOX]';
                    } else if (role === 'menu') {
                        return '[MENU]';
                    }
                    return '';
                }
                
                // Remove script and style elements
                const scripts = document.querySelectorAll('script, style');
                scripts.forEach(el => el.remove());
                
                // Process the body content recursively
                function processNode(node) {
                    if (node.nodeType === Node.TEXT_NODE) {
                        return node.textContent.trim();
                    }
                    
                    if (node.nodeType === Node.ELEMENT_NODE) {
                        const tag = getElementInfo(node);
                        let result = '';
                        
                        // Add opening tag if it's a special element
                        if (tag) {
                            result += tag + ' ';
                        }
                        
                        // Process children
                        for (const child of node.childNodes) {
                            result += processNode(child);
                        }
                        
                        return result + ' ';
                    }
                    
                    return '';
                }
                
                return processNode(document.body).replace(/\s+/g, ' ').trim();
            }
        """)

<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/xq/4rym6br90_9csp4rzkvd9r200000gn/T/ipykernel_64103/2506000108.py:1: SyntaxWarning: invalid escape sequence '\s'
  text_content = await page.evaluate("""


In [16]:
print(text_content)

[HEADER] [LINK] Skip to main content [LINK] We gratefully acknowledge support from the Simons Foundation,[LINK] member institutions , and all contributors.[LINK] Donate [LINK] [FORM] [INPUT:text] [LINK] Help |[LINK] Advanced Search [SELECT] All fields Title Author Abstract Comments Journal reference ACM classification MSC classification Report number arXiv identifier DOI ORCID arXiv author ID Help pages Full text [INPUT:hidden] [BUTTON] Search [LINK] Login [H1] Showing 1–50 of 1,313 results for all:AI regulation [LINK] Search v0.5.6 released 2020-02-24 [FORM] Search term or terms [INPUT:text] Field [SELECT] All fields Title Author(s) Abstract Comments Journal reference ACM classification MSC classification Report number arXiv identifier DOI ORCID License (URI) arXiv author ID Help pages Full text [BUTTON] Search [INPUT:radio] Show abstracts [INPUT:radio] Hide abstracts [LINK] Advanced Search [INPUT:hidden] [INPUT:hidden] [FORM] [SELECT] All fields Title Author(s) Abstract Comments Jour